In [1]:
# SUBROUTINE: given n divides adelic-level (N), find G(n)<GL2(Z/nZ)

%run utilities.ipynb

def reduction_mod_n(gens, N, n):
    """Reduction modulo n with enhanced caching and early exits."""
    cache_key = (tuple(tuple(g) if isinstance(g, (list, tuple)) else _mat_key(g) for g in gens), N, n)
    if cache_key in _reduction_cache:
        return _reduction_cache[cache_key]
    
    N = Integer(N); n = Integer(n)
    if N % n != 0:
        raise ValueError("Need n | N for the canonical reduction Z/NZ -> Z/nZ.")

    RN = _R(N)
    Rn = _R(n)

    # Convert generators to matrices over Z/NZ
    matsN = [ _to_mat_over_R(g, RN) for g in gens ]
    
    # Reduce modulo n more efficiently
    matsn = []
    for M in matsN:
        entries = M.list()
        reduced_entries = [ (_lift_int(x) % n) for x in entries ]
        matsn.append(matrix(Rn, 2, 2, reduced_entries))

    # Create subgroup and get cleaned generators
    G = GL(2, Rn).subgroup(matsn)
    clean = list(G.gens())
    clean_flat = []
    
    for g in clean:
        if hasattr(g, "matrix"):
            g = g.matrix()
        clean_flat.append([int(_lift_int(x)) for x in g.list()])
    
    # Cache result with size limit to prevent memory bloat
    if len(_reduction_cache) < 10000:  # Limit cache size
        _reduction_cache[cache_key] = clean_flat
    elif len(_reduction_cache) % 1000 == 0:  # Clear cache periodically
        _reduction_cache.clear()
        _reduction_cache[cache_key] = clean_flat
    
    return clean_flat